In [1]:
from plyer import notification
from datetime import datetime
import numpy as np
import pandas as pd
import requests

In [2]:
"""
___________________________________________________________________________________________
Função que recebe argumentos e retorna uma mensagem de erro ao usuário do seguinte formato:
| ATENÇÃO: Alerta Baixo/Médio/Alto (conforme input nivel)
| Falha no carregamento da base (input base) na etapa (input etapa).
| 2023-07-27 21:27:43.336652 (timestamp atual)
"""
def alerta (nivel, base, etapa):
    ## Dependendo do nível indicado na variável, selecionamos qual o nível do alerta retornado
    match nivel:
        case '1':
            nivel_alerta = 'Baixo'
        case '2':
            nivel_alerta = 'Médio'
        case '3':
            nivel_alerta = 'Alto'
        case _:
            ## Caso nível de alerta não esperado, retorna mensagem de erro
            return 'Nível selecionado incorreto!'
    
    timstamp_atual = datetime.now().strftime('%d/%m/%Y %H:%M:%S.%f')
    
    ## Chama função para notificar user, conforme variáveis inputadas
    notification.notify (
        title = (f'ATENÇÃO: Alerta {nivel_alerta}'),
        message = (f'Falha no carregamento da base {base} na etapa {etapa}.\n{timstamp_atual}'),
        app_name = ('Teste'), 
        timeout = 10
    )
    return

"""
___________________________________________________________________________________________
Funções que utilizam a API Brasil para extrair diversos dados conforme descritos abaixo:

» carregar_dados_estados
Traz a sigla, ID, nome e região de todos os estados do Brasil e retorna em um dataframe
"""
def carregar_dados_estados():
    url = 'https://brasilapi.com.br/api/ibge/uf/v1'
    req_UFs = requests.get(url)
    if req_UFs.status_code == 200:
        dados_UFs = pd.DataFrame(req_UFs.json())
        return(dados_UFs)
    else:
        alerta('3', 'estados do Brasil', 'carregamento de dados')
        return

"""
» carregar_dados_cidades
Utiliza argumento para trazer os nomes de todas as cidades do estado desejado, junto com o seu código IBGE
"""
def carregar_dados_cidades(estado_desejado):
    url = (f'https://brasilapi.com.br/api/ibge/municipios/v1/{estado_desejado}?providers=gov')
    req_cidades = requests.get(url)

    if req_cidades.status_code == 200:
        infos_cidades = pd.DataFrame(req_cidades.json())
        return(infos_cidades)
    else:
        alerta('3', 'cidades do estado {estado_desejado}', 'carregamento de dados')
        return

"""
» carregar_info_DDDs
Utiliza argumento para trazer todas as cidades e estados que utilizam o código de DDD desejado
"""
def carregar_info_DDDs(cod_DDD):
    url = (f'https://brasilapi.com.br/api/ddd/v1/{cod_DDD}')
    req_DDD = requests.get(url)

    if req_DDD.status_code == 200:
        infos_DDD = pd.DataFrame(req_DDD.json())
        return(infos_DDD)

    elif req_DDD.status_code == 404:
        return('DDD não encontrado')
        
    else:
        alerta('3', 'informação do DDD {cod_DDD}', 'carregamento de dados')
        return

In [3]:
# Testa o carregamento da base dos estados brasileiros
df_brasil = carregar_dados_estados()
df_brasil.head()

,id,sigla,nome,regiao
0,11,RO,Rondônia,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
1,12,AC,Acre,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
2,13,AM,Amazonas,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
3,14,RR,Roraima,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"
4,15,PA,Pará,"{'id': 1, 'sigla': 'N', 'nome': 'Norte'}"


In [5]:
# Testa o carregamento da base de cidades do estado de SP
df_SP = carregar_dados_cidades('SP')
df_SP.head()

,nome,codigo_ibge
0,ADAMANTINA,3500105
1,ADOLFO,3500204
2,AGUAÍ,3500303
3,ÁGUAS DA PRATA,3500402
4,ÁGUAS DE LINDÓIA,3500501


In [7]:
# Testa o carregamento da base de cidades que contém o DDD 11
df_DDD_11 = carregar_info_DDDs('11')
df_DDD_11.head()

,state,cities
0,SP,EMBU
1,SP,VÁRZEA PAULISTA
2,SP,VARGEM GRANDE PAULISTA
3,SP,VARGEM
4,SP,TUIUTI


In [8]:
# Cria uma base de dados de todos os DDDs nacionais
# Obs: Foi detectado que a API retorna erro caso formos puxar a base completa, portanto estamos parando no DDD 83
df_DDD = pd.DataFrame(columns = ['Estado', 'Cidade', 'DDD'])
DDDs_validos = ['11', '12', '13', '14', '15', '16', '17', '18', '19', '21', '22', '24', '27', '28', '31', '32', '33', '34', '35', '37', '38', '41', '42', '43', '44', '45', '46', '47', '48', '49', '51', '53', '54', '55', '61', '62', '63', '64', '65', '66', '67', '68', '69', '71', '73', '74', '75', '77', '79', '81', '82', '83']#, '84', '85', '86', '87', '88', '89', '91', '92', '93', '94', '95', '96', '97', '98', '99']
for i in DDDs_validos:
    df_aux = carregar_info_DDDs(i)
    if type(df_aux) == tuple:
        df_aux
        pass
    else:
        df_aux
        df_aux['DDD'] = i
        df_aux.columns = ['Estado', 'Cidade', 'DDD']
        df_DDD = pd.concat([df_DDD, df_aux], ignore_index = True)
    
df_DDD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4451 entries, 0 to 4450
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Estado  4451 non-null   object
 1   Cidade  4451 non-null   object
 2   DDD     4451 non-null   object
dtypes: object(3)
memory usage: 104.4+ KB
